In [1]:
library(tidyverse)
library(DBI)
library(RSQLite)

Warning message:
"package 'tidyverse' was built under R version 3.6.2"-- Attaching packages --------------------------------------- tidyverse 1.3.0 --
v ggplot2 3.2.1     v purrr   0.3.3
v tibble  2.1.3     v dplyr   0.8.4
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0
Warning message:
"package 'ggplot2' was built under R version 3.6.2"Warning message:
"package 'tidyr' was built under R version 3.6.2"Warning message:
"package 'dplyr' was built under R version 3.6.2"Warning message:
"package 'forcats' was built under R version 3.6.3"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter()  masks stats::filter()
x purrr::flatten() masks jsonlite::flatten()
x dplyr::lag()     masks stats::lag()
Warning message:
"package 'DBI' was built under R version 3.6.2"Warning message:
"package 'RSQLite' was built under R version 3.6.3"

#### 1. Se connecter à la base `SQLitefootball.sqlite`

In [2]:
db <- dbConnect(SQLite(), dbname = "football.sqlite")

#### 2. Lister les bases

In [3]:
dbListTables(db)

[1] "Country"           "League"            "Match"            
[4] "Player"            "Player_Attributes" "Team"             
[7] "Team_Attributes"   "sqlite_sequence"

#### 3. Afficher toutes les premières divisions présentes dans cette base de donnée (ligue 1 en France, sérieA en Italie etc...)
La table **League** contient uniquement les premières divisions dont les `id` et les `country_id` renvoient aux `id` des pays dans **Country**.

In [4]:
query <- "SELECT Pays.Name AS Pays, Ligue.Name AS Ligue
          FROM Country AS Pays JOIN League AS Ligue ON Pays.id = Ligue.country_id"

as_tibble(
    dbGetQuery(db, query)
)

Pays,Ligue
Belgium,Belgium Jupiler League
England,England Premier League
France,France Ligue 1
Germany,Germany 1. Bundesliga
Italy,Italy Serie A
Netherlands,Netherlands Eredivisie
Poland,Poland Ekstraklasa
Portugal,Portugal Liga ZON Sagres
Scotland,Scotland Premier League
Spain,Spain LIGA BBVA


#### 4. Afficher les champs de la table **Match**

In [5]:
dbListFields(db, "Match")

[1] "id"               "country_id"       "league_id"       
  [4] "season"           "stage"            "date"            
  [7] "match_api_id"     "home_team_api_id" "away_team_api_id"
 [10] "home_team_goal"   "away_team_goal"   "home_player_X1"  
 [13] "home_player_X2"   "home_player_X3"   "home_player_X4"  
 [16] "home_player_X5"   "home_player_X6"   "home_player_X7"  
 [19] "home_player_X8"   "home_player_X9"   "home_player_X10" 
 [22] "home_player_X11"  "away_player_X1"   "away_player_X2"  
 [25] "away_player_X3"   "away_player_X4"   "away_player_X5"  
 [28] "away_player_X6"   "away_player_X7"   "away_player_X8"  
 [31] "away_player_X9"   "away_player_X10"  "away_player_X11" 
 [34] "home_player_Y1"   "home_player_Y2"   "home_player_Y3"  
 [37] "home_player_Y4"   "home_player_Y5"   "home_player_Y6"  
 [40] "home_player_Y7"   "home_player_Y8"   "home_player_Y9"  
 [43] "home_player_Y10"  "home_player_Y11"  "away_player_Y1"  
 [46] "away_player_Y2"   "away_player_Y3"   "away_player_Y4"  
 [49] "away_player_Y5"   "away_player_Y6"   "away_player_Y7"  
 [52] "away_player_Y8"   "away_player_Y9"   "away_player_Y10" 
 [55] "away_player_Y11"  "home_player_1"    "home_player_2"   
 [58] "home_player_3"    "home_player_4"    "home_player_5"   
 [61] "home_player_6"    "home_player_7"    "home_player_8"   
 [64] "home_player_9"    "home_player_10"   "home_player_11"  
 [67] "away_player_1"    "away_player_2"    "away_player_3"   
 [70] "away_player_4"    "away_player_5"    "away_player_6"   
 [73] "away_player_7"    "away_player_8"    "away_player_9"   
 [76] "away_player_10"   "away_player_11"   "goal"            
 [79] "shoton"           "shotoff"          "foulcommit"      
 [82] "card"             "cross"            "corner"          
 [85] "possession"       "B365H"            "B365D"           
 [88] "B365A"            "BWH"              "BWD"             
 [91] "BWA"              "IWH"              "IWD"             
 [94] "IWA"              "LBH"              "LBD"             
 [97] "LBA"              "PSH"              "PSD"             
[100] "PSA"              "WHH"              "WHD"             
[103] "WHA"              "SJH"              "SJD"             
[106] "SJA"              "VCH"              "VCD"             
[109] "VCA"              "GBH"              "GBD"             
[112] "GBA"              "BSH"              "BSD"             
[115] "BSA"

#### 5. Extraire dans un data-frame les résultats de la Ligue 1 française
Utilisation d'une sous-requête au lieu d'un jointure pour esquiver la gestion de `.name_repair` lors de la conversion en tibble.

In [6]:
query1 <- "SELECT *
           FROM Match
           WHERE league_id = (
            SELECT id
            FROM League
            WHERE Name = 'France Ligue 1'
           )"

matchFR <- as_tibble(
    dbGetQuery(db, query1)
)

matchFR

id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
4769,4769,4769,2008/2009,1,2008-08-09 00:00:00,483129,8583,9830,2,...,3.80,2.00,2.90,3.75,2.10,3.00,3.75,2.00,2.88,3.75
4770,4769,4769,2008/2009,1,2008-08-09 00:00:00,483130,9827,7819,2,...,7.50,1.60,3.30,5.50,1.57,3.60,6.00,1.50,3.50,6.50
4771,4769,4769,2008/2009,1,2008-08-09 00:00:00,483131,9746,9831,1,...,3.10,2.30,2.90,3.00,2.35,2.90,3.20,2.38,2.88,2.88
4772,4769,4769,2008/2009,1,2008-08-09 00:00:00,483132,8682,8689,0,...,4.00,2.00,2.90,3.75,2.00,3.10,3.75,1.91,3.00,4.00
4773,4769,4769,2008/2009,1,2008-08-10 00:00:00,483133,9748,9941,3,...,12.00,1.30,4.00,10.00,1.36,4.25,9.00,1.30,4.33,9.00
4774,4769,4769,2008/2009,1,2008-08-09 00:00:00,483134,9829,9847,1,...,3.20,2.25,3.00,3.00,2.38,3.00,3.00,2.30,3.00,2.88
4775,4769,4769,2008/2009,1,2008-08-09 00:00:00,483135,8481,8639,0,...,3.80,2.10,3.00,3.30,2.20,3.00,3.30,2.00,3.00,3.60
4776,4769,4769,2008/2009,1,2008-08-09 00:00:00,483136,9851,8592,4,...,2.88,2.65,2.90,2.65,2.70,3.00,2.60,2.62,3.00,2.50
4777,4769,4769,2008/2009,1,2008-08-09 00:00:00,483137,9874,9855,1,...,4.33,1.85,3.10,4.20,1.85,3.20,4.25,1.83,3.10,4.20
4778,4769,4769,2008/2009,1,2008-08-09 00:00:00,483138,9873,9853,1,...,2.70,2.25,2.90,3.10,2.50,2.90,2.90,2.40,2.88,2.88


#### 6. Extraire dans un data-frame les résultats de la Ligue 1 française pour l’année 2015-2016 et ajouter le score du match et les points pour chaque match (match perdu = 0 pt, match nul = 1 pt et match gagné = 3 points ) de la saison 2015-2016.
On ne gardera que les colonnes qui sont pas utiles pour la suite.

In [7]:
matchFR <- matchFR %>% 
filter(season == "2015/2016") %>%
mutate(
    score = home_team_goal - away_team_goal,
    home_points = case_when(
        score < 0 ~ 0,
        score > 0 ~ 3,
        TRUE ~ 1
    )
) %>% 
select(home_team_api_id, home_team_goal, home_points, away_team_api_id, away_team_goal, score, season)

matchFR

home_team_api_id,home_team_goal,home_points,away_team_api_id,away_team_goal,score,season
7794,2,3,9851,1,1,2015/2016
9827,1,0,9837,2,-1,2015/2016
8639,0,0,9847,1,-1,2015/2016
9748,0,1,8689,0,0,2015/2016
8592,0,0,7819,1,-1,2015/2016
10249,0,0,8121,2,-2,2015/2016
9830,1,3,9747,0,1,2015/2016
9831,1,0,9829,2,-1,2015/2016
9941,2,3,9853,1,1,2015/2016
10242,0,1,6391,0,0,2015/2016


#### 7. Trouver la meilleure équipe à domicile (celle qui le plus de points en ne comptant que les match à domicile)
Importation de la table des équipes *(pour obtenir le nom)*.

In [8]:
team <<- as_tibble(tbl(db, "Team"))

Création d'une fonction qui sera utilisée dans les 3 prochaines questions.

In [9]:
best_n <- function(tb, n){
    #' Renvoie un tibble des n meilleurs équipes par points
    #' tb: tibble ayant les attributs de matchFR
    #' n: entier
    
    best <- tb %>%
    group_by(home_team_api_id) %>%
    summarise(total_points = sum(home_points)) %>% 
    arrange(desc(total_points)) %>% 
    top_n(n, total_points) %>%
    inner_join(
        team,
        by = c(home_team_api_id = "team_api_id")
    ) %>% 
    select(best_team = team_long_name, total_points)
    
    return(best)
}

La meilleure équipe à domicile.

In [10]:
matchFR %>%
best_n(1)

best_team,total_points
Paris Saint-Germain,48


#### 8. Trouver ensuite les trois meilleures équipes du championnat 2015-2016.
Ajout des points gagnés en étant visiteur lors du championnat.

In [11]:
match <- as_tibble(tbl(db, "Match")) %>% 
filter(season == "2015/2016") %>%
mutate(
    score = home_team_goal - away_team_goal,
    home_points = case_when(
        score < 0 ~ 0,
        score > 0 ~ 3,
        TRUE ~ 1
    ),
    away_points = case_when(
        score > 0 ~ 0,
        score < 0 ~ 3,
        TRUE ~ 1
    )
) %>% 
select(home_team_api_id, home_team_goal, home_points, away_team_api_id, away_team_goal, away_points, score)

Création d'un vecteur de noms communs pour faciliter l'aggrégation des points gagnés à domicile et en tant que visiteur.

In [12]:
newnames <- colnames(match)[4:6]
names(newnames) <- colnames(match)[1:3]

Les 3 meilleurs équipes du championnat en terme de points.

In [13]:
match %>%
select(all_of(newnames)) %>% 
bind_rows(match[,1:3]) %>% 
best_n(3)

best_team,total_points
Paris Saint-Germain,96
FC Barcelona,91
Juventus,91


<h4> 9. Trouver les trois meilleures équipes si nous ajoutons à l’attribution des points les bonus suivants :
    <ul>
    <li> bonus offensif : +1 point si gain d’un match avec un ecart de 2 buts ou plus ;</li>
    <li> bonus spectacle : +1 point si le match est perdu mais l’équipe a quand même marqué 2 buts ou plus.</li>
    </ul>
</h4>

Ajout des bonus, s'il doit y en avoir, et affichage des 3 meilleurs équipes.

In [14]:
match %>% 
mutate(
    home_points = if_else(
        score >= 2 | (score < 0 & home_team_goal >= 2),
        home_points +1, home_points
    ),
    away_points = if_else(
        score <= -2 | (score > 0 & away_team_goal >= 2),
        away_points +1, away_points
    )
) %>% 
select(all_of(newnames)) %>% 
bind_rows(.[,1:3]) %>% 
best_n(3)

best_team,total_points
Paris Saint-Germain,114
PSV,106
Sporting CP,104
